In [1]:
from typing import Callable
from pydantic import BaseModel, field_validator, ValidationError
import re
import warnings
warnings.filterwarnings('ignore')

In [3]:
class TwitterPostModel(BaseModel):
    username: str
    post: str

    @field_validator('username')
    def validate_twitter_username(cls, v):
        if not v.startswith('@'):
            raise ValueError("Twitter username must start with '@'")
        if len(v) < 2 or len(v) > 16:  # '@' + up to 15 chars
            raise ValueError("Twitter username must include '@' and 1–15 valid characters")
        if not re.match(r'^@[A-Za-z0-9_]+$', v):
            raise ValueError("Twitter username may contain only letters, numbers, and underscores after '@'")
        return v

    @field_validator('post')
    def validate_twitter_post(cls, v):
        if not v or len(v) > 280:
            raise ValueError("Twitter post must be non-empty and at most 280 characters")
        return v


def twitter_post_validator(func: Callable[[str, str], None]) -> Callable[[str, str], None]:
    def wrapper(username: str, post: str):
        try:
            TwitterPostModel(username=username, post=post)
        except ValidationError as e:
            raise ValueError(f"Invalid Twitter data: {e}")
        return func(username, post)
    return wrapper


@twitter_post_validator
def print_twitter_post(username: str, post: str):
    print(f"Valid Twitter username: {username}")
    print(f"Post: {post}")


if __name__ == "__main__":
    try:
        print("Example of a successful post")
        print_twitter_post("@olami", "I am one of the best graduating student from GEN AI FELLOWSHIP as an AI Engineer.")
    except ValueError as e:
        print("Error:", e)
        
        
    print("\nExample of a failing post")
    try:
        print_twitter_post("sammy!", "This will fail because username lacks @ or has invalid chars")
    except ValueError as e:
        print("Error:", e)


Example of a successful post
Valid Twitter username: @olami
Post: I am one of the best graduating student from GEN AI FELLOWSHIP as an AI Engineer.

Example of a failing post
Error: Invalid Twitter data: 1 validation error for TwitterPostModel
username
  Value error, Twitter username must start with '@' [type=value_error, input_value='sammy!', input_type=str]
    For further information visit https://errors.pydantic.dev/2.8/v/value_error
